<a href="https://colab.research.google.com/github/tolgakurtuluss/HacktoberFest/blob/main/aiornot_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminaries

In [1]:
!pip install -q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.9 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) N
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!git config --global credential.helper store

In [3]:
from huggingface_hub import hf_hub_download
from shutil import unpack_archive
from pathlib import Path
import pandas as pd

train_zip_file = hf_hub_download('competitions/aiornot', 'train.zip', repo_type='dataset', revision='b843a82bd712648b2fe0dc043cf8a04475491d38')
unpack_archive(train_zip_file, './')

test_zip_file = hf_hub_download('competitions/aiornot', 'test.zip', repo_type='dataset', revision='b843a82bd712648b2fe0dc043cf8a04475491d38')
unpack_archive(test_zip_file, './')

dataframe = pd.read_csv(hf_hub_download('competitions/aiornot', 'train.csv', repo_type='dataset', revision='b843a82bd712648b2fe0dc043cf8a04475491d38'))

## Data splitting

In [4]:
dataframe.head()

,id,label
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,0
4,4.jpg,1


In [5]:
dataframe["label"].value_counts()

1    10330
0     8288
Name: label, dtype: int64

In [161]:
from sklearn.model_selection import train_test_split 

seed= 42
train_df, validation_df = train_test_split(
    dataframe, test_size=0.1, stratify=dataframe["label"].values, random_state=seed
)
validation_df, holdout_df = train_test_split(
    validation_df, test_size=0.5, stratify=validation_df["label"].values, random_state=seed
)

train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)
holdout_df = holdout_df.reset_index(drop=True)

train_df.shape, validation_df.shape, holdout_df.shape

((16756, 2), (931, 2), (931, 2))

In [162]:
train_df["image_path"] = train_df["id"].apply(lambda x: "train/" + x)
validation_df["image_path"] = validation_df["id"].apply(lambda x: "train/" + x)
holdout_df["image_path"] = holdout_df["id"].apply(lambda x: "train/" + x)

In [163]:
from imutils import paths
import pandas as pd 

test_image_paths = list(paths.list_images("test"))
final_test_df = pd.DataFrame()
final_test_df["image_path"] = test_image_paths
final_test_df["label"] = 0
final_test_df.head()

,image_path,label
0,test/14965.jpg,0
1,test/39665.jpg,0
2,test/706.jpg,0
3,test/17862.jpg,0
4,test/32337.jpg,0


## Data prep

In [164]:
import torch 
import PIL

class AIOrNotDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transforms, split):
        self.dataframe = dataframe
        self.transforms = transforms 
        self.split = split

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = self.dataframe.loc[idx, "image_path"]
        image = PIL.Image.open(img_path)
        image = self.transforms(image)
        
        if self.split != "test":
            label = self.dataframe.loc[idx, "label"] 
            return {"img_path": img_path, "image": image, "label": label} 
        else:
            return {"img_path": img_path, "image": image}

In [127]:
from transformers import AutoModelForImageClassification, AutoImageProcessor


label2id = {"no_ai": 0, "ai": 1}
id2label = {0: "no_ai", 1: "ai"}

#model_ckpt = "microsoft/beit-base-patch16-224-pt22k-ft22k" ++v3

model_ckpt = "microsoft/swinv2-base-patch4-window16-256"

model = AutoModelForImageClassification.from_pretrained(
    model_ckpt, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
image_processor = AutoImageProcessor.from_pretrained(model_ckpt)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--swinv2-base-patch4-window16-256/snapshots/628b75ababc4dad9f5bbabc1bf8bb612c4ab2f78/config.json
Model config Swinv2Config {
  "_name_or_path": "microsoft/swinv2-base-patch4-window16-256",
  "architectures": [
    "Swinv2ForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    18,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "no_ai",
    "1": "ai"
  },
  "image_size": 256,
  "initializer_range": 0.02,
  "label2id": {
    "ai": 1,
    "no_ai": 0
  },
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swinv2",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "pretrained_window_sizes": [
    0,
    0,
    0,
    0
  ],


In [165]:
from torchvision import transforms as T


mean, std = image_processor.image_mean, image_processor.image_std
size = image_processor.size["height"]

train_transforms = T.Compose([
    T.RandomResizedCrop(size),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])
test_transforms = T.Compose([
    #T.Resize(256),
    T.CenterCrop(size),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std)
])

In [166]:
train_dataset = AIOrNotDataset(train_df, train_transforms, split="train")
val_dataset = AIOrNotDataset(validation_df, test_transforms, split="validation")
holdout_dataset = AIOrNotDataset(holdout_df, test_transforms, split="holdout")
final_test_dataset = AIOrNotDataset(final_test_df, test_transforms, split="test")

In [167]:
sample_train = next(iter(train_dataset))
sample_train.keys(), sample_train["image"].size()

(dict_keys(['img_path', 'image', 'label']), torch.Size([3, 256, 256]))

## Prepare trainer

In [168]:
from transformers import TrainingArguments

model_name = model_ckpt.split("/")[-1]
batch_size = 16

args = TrainingArguments(
    f"{model_name}-finetuned-aiornot-test-1",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [169]:
from sklearn.metrics import accuracy_score, f1_score, log_loss

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    logloss = log_loss(labels, preds)
    return {"accuracy": acc, "f1": f1, "log_loss": logloss}

In [170]:
def collate_fn(examples):
    pixel_values = torch.stack([example["image"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [171]:
from transformers import Trainer 

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

Cloning https://huggingface.co/tkurtulus/swinv2-base-patch4-window16-256-finetuned-aiornot-test-1-finetuned-aiornot-test-1 into local empty directory.


KeyboardInterrupt: ignored

## Train!

In [135]:
train_results = trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16756
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 1310
  Number of trainable parameters = 86895866


Epoch,Training Loss,Validation Loss,Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.133200,1.187164,0.103375,0.965628,0.965282,23.076100,40.345000,2.557000
2,0.132800,0.927479,0.088516,0.973147,0.972753,22.985300,40.504000,2.567000
3,0.043200,1.521047,0.121425,0.955961,0.955681,23.037600,40.412000,2.561000
4,0.056000,0.741984,0.057579,0.978518,0.978197,22.966200,40.538000,2.569000


***** Running Evaluation *****
  Num examples = 931
  Batch size = 16
Saving model checkpoint to swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-262
Configuration saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-262/config.json
Model weights saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-262/pytorch_model.bin
Image processor saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-262/preprocessor_config.json
Image processor saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 931
  Batch size = 16
Saving model checkpoint to swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-524
Configuration saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-524/config.json
Model weights saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-524/pytorch_mod

Epoch,Training Loss,Validation Loss,Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.133200,1.187164,0.103375,0.965628,0.965282,23.076100,40.345000,2.557000
2,0.132800,0.927479,0.088516,0.973147,0.972753,22.985300,40.504000,2.567000
3,0.043200,1.521047,0.121425,0.955961,0.955681,23.037600,40.412000,2.561000
4,0.056000,0.741984,0.057579,0.978518,0.978197,22.966200,40.538000,2.569000
5,0.041700,0.853272,0.081990,0.975295,0.975052,23.072500,40.351000,2.557000


***** Running Evaluation *****
  Num examples = 931
  Batch size = 16
Saving model checkpoint to swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-1310
Configuration saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-1310/config.json
Model weights saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-1310/pytorch_model.bin
Image processor saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-1310/preprocessor_config.json
Image processor saved in swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from swinv2-base-patch4-window16-256-finetuned-aiornot-test-1/checkpoint-1048 (score: 0.9785177228786252).


In [ ]:
trainer.evaluate(holdout_dataset)

## Prepare submission file

In [172]:
test_loader = torch.utils.data.DataLoader(
    final_test_dataset, batch_size=256, pin_memory=True
)
device = "cuda" if torch.cuda.is_available() else "cpu"
_ = model.to(device)

In [173]:
file_paths = []
pred_ids = []

for batch in test_loader:
    image_paths = batch["img_path"]
    image_paths = [x.split("/")[-1] for x in image_paths]
    file_paths.extend(image_paths)
    
    images = batch["image"].to(device)
    inputs = {"pixel_values": images}
    
    with torch.no_grad(): 
        logits = model(**inputs).logits

    predictions = logits.argmax(-1).cpu().numpy().tolist()
    pred_ids.extend(predictions)

OutOfMemoryError: ignored

In [ ]:
submission_df = pd.DataFrame({"id": file_paths, "label": pred_ids})
submission_df.head()

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

submission_df.to_csv(f"{TIMESTAMP}.csv", index=False)

In [ ]:
from google.colab import files
files.download(f"{TIMESTAMP}.csv") 

In [156]:
import gc

model.cpu()
#del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#################################

from transformers import AutoFeatureExtractor, AutoModelForImageClassification

extractor = AutoFeatureExtractor.from_pretrained("tkurtulus/beit-base-patch16-224-pt22k-ft22k-finetuned-aiornot-test-3")

model = AutoModelForImageClassification.from_pretrained("tkurtulus/beit-base-patch16-224-pt22k-ft22k-finetuned-aiornot-test-3")